**jupyter notebook 환경설정 -> nbextensions 설치 (굳이 없어도 상관은 없으나 편리함)**
> [nbextensions 설치](https://hmiiing.tistory.com/entry/%EC%A3%BC%ED%94%BC%ED%84%B0-%EB%85%B8%ED%8A%B8%EB%B6%81%EC%97%90-%EC%9D%B5%EC%8A%A4%ED%85%90%EC%85%98-%EC%84%A4%EC%B9%98%ED%95%98%EA%B8%B0nbextention)

<br>

※ Windows 기준으로 작성되었음. 이해를 위해 파일 탐색기를 함께 켜 놓고 보는 것을 추천.

<br>

**공식 깃허브 wiki또한 참고할 것**
> [yolov5 공식 깃허브 wiki](https://github.com/ultralytics/yolov5/wiki)

##### 시작하기 전 확인사항
- anaconda 설치
- anaconda prompt 실행 시 관리자 모드로 실행
- anaconda 가상환경 만들기
- conda update conda, conda update --all
- [pytorch 라이브러리 설치](https://pytorch.org/)

# YOLOv5를 사용해 커스텀 학습하기

## train 전 설정

- 현재 Directory 확인, 작업할 폴더로 이동 (예시의 경우 C드라이브에 YOLOv5 폴더를 생성해 둠)

In [1]:
%pwd
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 8242-ECC2

 C:\OverwatchAimHackDetect 디렉터리

2022-05-09  오전 01:27    <DIR>          .
2022-05-09  오전 01:25             4,238 .gitignore
2022-05-09  오전 01:27    <DIR>          .ipynb_checkpoints
2022-05-09  오전 01:25            18,201 OverwatchCharacterDetect.ipynb
2022-05-09  오전 01:25             1,114 README.md
2022-05-09  오전 01:25    <DIR>          yolov5
               3개 파일              23,553 바이트
               3개 디렉터리  401,937,309,696 바이트 남음


- yolov5 깃 클론

<br>

※컴퓨터에 git이 깔려있지 않으면 실행 불가. 실행 전 반드시 설치할 것.
- 환경변수 설정할 것(시스템 path)
<br>

> [git 설치 링크](https://gitforwindows.org/)

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


- 깃 클론한 yolov5 폴더로 이동

In [3]:
%cd yolov5

C:\OverwatchAimHackDetect\yolov5


- 폴더 내에 있는 requirements.txt 기반 필수 라이브러리 설치
- torch 라이브러리 임포트, 버전 및 GPU 확인

In [ ]:
%pip install -qr requirements.txt 

#이건 anaconda prompt에서 실행하는 것이 좋음. (ctrl + c 두번으로 jupyter notebook 빠져나가기 가능.)

In [4]:
import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.11.0 _CudaDeviceProperties(name='GeForce RTX 3070', major=8, minor=6, total_memory=8192MB, multi_processor_count=46)


- 예시의 경우 라벨링 툴로 [roboflow](https://app.roboflow.com/) 사용
- roboflow에서 프로젝트 생성 및 데이터 라벨링 작업 후 YOLO v5 pytorch 포맷으로 export
- 생성된 명령어(다운로드 링크 포함)로 데이터 zip파일 다운로드

In [ ]:
#링크
download_link = "roboflow 사이트 참고"
#다운받을 zip파일 이름(필요 없으면 주석처리)
zipfile_name = '직접작성'

- 클론받은 yolov5 폴더 내에 datasets 폴더 생성, 경로 이동
- curl 명령어(roboflow에 있는 그대로 사용 가능)로 데이터 zip파일 다운로드

In [ ]:
!mkdir datasets
%cd datasets

!curl -L {download_link} > {zipfile_name}

- colab과 달리 windows에서는 unzip 명령, rm 명령 사용 불가
- 따라서 아래 shutil 라이브러리를 사용, 압축 풀기
- 압축을 푼 내용물을 현재 디렉터리로 복사(xcopy 명령, 옵션은 인터넷 참고)

In [ ]:
import shutil

filename = '직접작성'
extract_dir = '직접작성'
archive_format = "zip"

shutil.unpack_archive(filename, extract_dir, archive_format)

!xcopy {extract_dir} . /e /h /k /y

- 현재 디렉터리로 복사했으므로 압축 푼 폴더 및 압축파일 삭제(del, rmdir 명령)

In [ ]:
!del {filename}
!rmdir /s /q {extract_dir}

- glob 라이브러리 사용, train, valid 폴더에 있는 이미지 경로 주소 목록을 담은 txt 파일 생성

In [ ]:
from glob import glob

train_img_list = glob('./train/images/*.jpg') + glob('./train/images/*.jpeg')
valid_img_list = glob('./valid/images/*.jpg') + glob('./valid/images/*.jpeg')

with open('./train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
            
with open('./valid.txt', 'w') as f:
    f.write('\n'.join(valid_img_list) + '\n')

- data.yaml 파일 수정
- 코드 적지 않았음. 직접 코드를 작성해 내용을 변경하거나, 아니면 직접 yaml 파일 수정할 것.
    - train: 위에서 생성한 train.txt 경로
    - val: 위에서 생성한 valid.txt 경로

    - nc: 구분할 클래스 개수
    - names: ['클래스명1', '클래스명2', ...]

## 모델 학습시키기

- !python /content/yolov5/train.py로 train.py 파일을 실행시킨다.

<br>

**이 때 설정할 수 있는 properties 목록**

- img: define input image size
- batch: determine batch size
- epochs: define the number of training epochs. (Note: often, 3000+ are common here!)
- data: set the path to our yaml file
- cfg: specify our model configuration
- weights: specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive folder)
- name: result names
- nosave: only save the final checkpoint
- cache: cache images for faster training
- workers: max dataloader workers (per RANK in DDP mode)

<br>

* cfg에는 yolov5 n, s, m, l, x 중 어떤 모델을 선택해 학습할지 선택해 그에 맞는 yaml 파일 경로를 적어주면 된다.
* yolov5 개발자가 미리 학습한 모델 가중치를 가지고 학습하는 것을 추천한다. [가중치 다운로드 링크](https://github.com/ultralytics/yolov5/releases)
* yolov5 공식 깃허브의 학습 팁 참고해 학습시킬 것 [Tips for Best Training Results](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results)

<br>

**아래는 예시이므로 에포크 3회, s 모델로 학습하였음**

In [ ]:
# 파일 위치에 따라 코드 내용이 달라지므로(상대경로) 항상 현재 폴더 확인할 것
%cd C:\OverwatchAimHackDetect

In [ ]:
# jupyter notebook을 나가서 아래 명령을 실행하면 epoch 진행도 확인 가능. jupyter에서는 실행이 끝나야만 확인 가능하다.
!python ./yolov5/train.py --img 416 --batch 16 --epochs 3 --data ./yolov5/datasets/data.yaml --cfg ./yolov5/models/yolov5s.yaml --weights ./pretrainedModel/yolov5s6.pt --name test --cache --workers 0

- 학습시킬 때 지정한 이름인 test 폴더에 저장되었음. >> yolov5\runs\train\test

## 객체 검출하기

- detect.py로 학습된 모델 가중치인 best.pt를 가지고 동영상 또는 이미지에서 객체 인식 가능
    - conf : confidence. 0.1의 경우 confidence 값 0.1이상이면 화면에 바운딩 박스 표시
    - source : 객체 검출할 영상 경로

In [ ]:
!python ./yolov5/detect.py --weights ./yolov5/runs/train/test/weights/best.pt --conf 0.1 --source ./yolov5/datasets/test/test.mp4

yolov5/runs/detect/ 의 exp 폴더에서 결과 이미지 확인 가능
- 위 결과를 확인하면 아무것도 검출해내지 못한 것을 확인할 수 있으나, 이것은 예시이기 때문임.

# 참고

## datasets 폴더에 데이터를 더 추가했을 때

roboflow에서 라벨링을 한다면 라벨 txt 파일이 생성되는데, 다른 프로젝트에서 작업한 내용을 export해서 그대로 추가해 작업하면 문제가 생김.
- dataset A, dataset B가 있다고 할 때, A는 1과 2, B는 3과 4라는 클래스를 구분하고 있다고 하자.
- 이때 컴퓨터에서 미리 다운받아 둔 datasets A에 B 내용을 추가하려고 한다.
- A는 1과 2라는 클래스를 0, 1로 구분함.
- B도 3과 4라는 클래스를 0, 1로 구분함.
- 따라서 라벨 txt 파일 내용을 변경해 줄 필요가 있음.
- txt 파일을 열면 띄워쓰기로 구분된 숫자들 중 가장 첫번째 숫자가 클래스를 나타냄. 이것을 변경해 주자.

In [ ]:
# train 폴더 내에 몇개의 라벨파일이 있나 세는 코드
from glob import glob
import os
train_label_list = glob('./train/labels/*.txt')
count = 0
for path in train_label_list:
    print(path)
    count = count + 1

print(count)

In [ ]:
# train 폴더 내 label 파일 내용을 바꿔주는 코드 (아래 예시의 경우 0을 1로 바꿔준다.)
from glob import glob 
import os 
train_label_list = glob('./train/labels/*.txt')
for path in train_label_list:
    print(path)
    f = open(path, "rt")
    newText = f.read()
    if newText.startswith("0"):
        f2 = open(path, 'wt')
        f2.write(newText.replace(newText[0], '1', 1))
        f2.close()
    f.close()

In [ ]:
# 위 코드와 마찬가지. valid 폴더 내 label 파일 내용을 바꿔주는 코드 (아래 예시의 경우 0을 1로 바꿔준다.)
from glob import glob 
import os 
train_label_list = glob('./valid/labels/*.txt')
for path in train_label_list:
    print(path)
    f = open(path, "rt")
    newText = f.read()
    if newText.startswith("0"):
        f2 = open(path, 'wt')
        f2.write(newText.replace(newText[0], '1', 1))
        f2.close()
    f.close()

## 학습 실패 시 대처방법

- **dataloader.py** 파일 _try_get_data에서 아래와 같은 문제가 발생한다고 출력
    - data = self._data_queue.get(timeout=timeout)

> --workers 인자가 포함되어있는지 확인, --workers 0으로 설정 [링크 참고](https://github.com/ultralytics/yolov5/issues/1675)

- 위 에러를 해결했음에도 아래와 같은 이유로 학습이 되지 않을 때
    - **OMP: Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.** 에러

> 라이브러리 중복 문제, 아래 코드 추가
>>```
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
```